In [30]:
import LinearAlgebra
import Random
using Printf
using LinearAlgebra

function generate_matrix(l::Float64, r::Float64, n::Int)
    return rand(n, n) .* (r - l) .+ l
end

function generate_vector(l::Float64, r::Float64, n::Int)
    return rand(n) .* (r - l) .+ l
end

function make_strictly_diagonally_dominant(A::AbstractMatrix; margin::Float64=1.0)
    n, m = size(A)
    @assert n == m "Матрица должна быть квадратной"
    Ā = copy(Matrix{Float64}(A))
    for i in 1:n
        row_sum_off = sum(abs, Ā[i, :]) - abs(Ā[i, i])
        Ā[i, i] = sign(Ā[i,i] == 0 ? 1.0 : Ā[i,i]) * (row_sum_off + margin)
    end
    return Ā
end

function has_diagonal_dominance(A::AbstractMatrix)
    n, m = size(A); @assert n == m "Матрица должна быть квадратной"
    for i in 1:n
        s = sum(abs, A[i, :]) - abs(A[i,i])
        if abs(A[i,i]) < s
            return false
        end
    end
    return true
end

function split_LDU(A::AbstractMatrix)
    n, m = size(A); @assert n == m "Матрица должна быть квадратной"
    L = Matrix(tril(A, -1))
    D = Diagonal(diag(A))
    U = Matrix(triu(A,  1))
    return L, D, U
end

function jacobi(A::AbstractMatrix, b::AbstractVector; x0=nothing,
                tol::Float64=1e-8, maxiter::Int=100_000)
    n, m = size(A); @assert n == m == length(b)
    x = x0 === nothing ? zeros(Float64, n) : copy(x0)
    L, D, U = split_LDU(A)
    Dinv = Diagonal(1.0 ./ diag(D))
    P = -(Dinv * (L + U))
    g = Dinv * b
    for k in 1:maxiter
        xnew = P * x + g
        inc = norm(xnew - x) / max(1.0, norm(xnew))
        x = xnew
        if inc < tol
            return (x, k, inc)
        end
    end
    return (x, maxiter, norm(P*x + g - x))
end

function seidel(A::AbstractMatrix, b::AbstractVector; x0=nothing,
                     tol::Float64=1e-8, maxiter::Int=100_000)
    n, m = size(A); @assert n == m == length(b)
    x = x0 === nothing ? zeros(eltype(b), n) : copy(x0)
    L = Matrix(tril(A, -1)); D = Diagonal(diag(A)); U = Matrix(triu(A, 1))
    DL = LowerTriangular(D + L)
    for k in 1:maxiter
        xnew = DL \ (b - U * x)
        inc = norm(xnew - x) / max(1.0, norm(xnew))
        x = xnew
        if inc < tol
            return (x, k, inc)
        end
    end
    return (x, maxiter, norm(DL \ (b - U * x) - x))
end

function seidel_solver(A::AbstractMatrix, b::AbstractVector; x0=nothing,
                                tol::Float64=1e-8, maxiter::Int=100_000)
    x, k, inc = seidel(A, b; x0=x0, tol=tol, maxiter=maxiter)
    return (x, k, inc)
end

time_s() = time_ns() / 1e9

function run_and_time(solver, A, b; kwargs...)
    t0 = time_s()
    out = solver(A, b; kwargs...)
    t1 = time_s()
    x, k, res = out[1], out[2], out[3]
    return (x=x, k=k, res=res, t=t1 - t0)
end

function print_matrix(A; name::AbstractString="A", preview::Int=6, print_full::Bool=false)
    n, m = size(A)
    println("$name: размер $n×$m")
    if print_full || (n <= preview && m <= preview)
        show(stdout, "text/plain", A); println()
    else
        p = min(preview, n, m)
        println("Первые $p×$p элементы:")
        show(stdout, "text/plain", A[1:p, 1:p]); println()
    end
end

function print_vector(v; name::AbstractString="b", preview::Int=10, print_full::Bool=false)
    n = length(v)
    println("$name: длина $n")
    if print_full || n <= preview
        show(stdout, "text/plain", v); println()
    else
        p = min(preview, n)
        println("Первые $p элементов:")
        show(stdout, "text/plain", v[1:p]); println()
    end
end

function compare_run(n::Int=5; l::Float64=-1.0, r::Float64=1.0,
                     tol::Float64=1e-10, maxiter::Int=200_000,
                     seed::Union{Int,Nothing}=nothing,
                     print_data::Bool=true, preview::Int=6, print_full::Bool=false)

    if seed !== nothing
        Random.seed!(seed)
    end

    Araw = generate_matrix(l, r, n)
    A = make_strictly_diagonally_dominant(Araw; margin=1.0)
    b = generate_vector(l, r, n)
    x0 = zeros(n)

    if print_data
        println("Данные задачи")
        print_matrix(A; name="A", preview=preview, print_full=print_full)
        print_vector(b; name="b", preview=max(10, preview), print_full=print_full)
        println()
    end

    jac = run_and_time(jacobi, A, b; x0=x0, tol=tol, maxiter=maxiter)
    gs  = run_and_time(seidel_solver, A, b; x0=x0, tol=tol, maxiter=maxiter)

    println("Сравнение методов")
    @printf("%-16s  %-12s  %-12s  %-12s\n", "Метод", "Итераций", "Приращение", "Время, с")
    @printf("%-16s  %-12d  %-12.3e  %-12.6f\n", "Якоби",   jac.k, jac.res, jac.t)
    @printf("%-16s  %-12d  %-12.3e  %-12.6f\n", "Зейдель", gs.k,  gs.res,  gs.t)
    println()

    return nothing
end


function main(; n::Int=100, l::Float64=-1.0, r::Float64=1.0,
               tol::Float64=1e-10, maxiter::Int=200_000,
               seed::Union{Int,Nothing}=nothing,
               print_data::Bool=true, preview::Int=6, print_full::Bool=false)
    compare_run(n; l=l, r=r, tol=tol, maxiter=maxiter, seed=seed,
                print_data=print_data, preview=preview, print_full=print_full)
end

main()


Данные задачи
A: размер 100×100
Первые 6×6 элементы:
6×6 Matrix{Float64}:
 49.3513      -0.821549   -0.807574  -0.609557    0.768537  -0.548979
 -0.394013    49.6394     -0.271208   0.288782   -0.685713  -0.97255
  0.00430311   0.447136  -52.1138    -0.98236    -0.074886   0.536784
  0.674579     0.60136     0.873915  52.7108      0.347897  -0.516475
  0.0771567   -0.420051    0.65027    0.430504  -47.938     -0.610576
  0.00445068  -0.913021    0.144843   0.515757   -0.633382  44.6796
b: длина 100
Первые 10 элементов:
10-element Vector{Float64}:
  0.6618451980171207
 -0.6007618151054841
  0.13164737679293492
 -0.5663951934557674
 -0.5492480586038608
 -0.10474061066308615
 -0.6717929848846778
 -0.5677441779982335
 -0.12847709617647585
 -0.803477366343758

Сравнение методов
Метод             Итераций      Приращение    Время, с    
Якоби             11            3.243e-11     0.114111    
Зейдель           9             1.013e-11     0.000097    

